# Notebook to create `expressions` module

In [1]:
from pathlib import Path

PACKAGE = "mintalib"
ROOTDIR = Path.cwd().parent
PKGDIR = ROOTDIR.joinpath(f"src/{PACKAGE}").resolve(strict=True)

if '__file__' in globals():
    print(f"Running {__file__} ...")


In [2]:
import polars as pl
from mintalib.samples import sample_prices

prices = sample_prices()
prices = pl.from_pandas(prices, include_index=True, nan_to_null=True)
prices


date,open,high,low,close,volume
datetime[ns],f64,f64,f64,f64,i64
1980-12-12 00:00:00,0.098943,0.099373,0.098943,0.098943,469033600
1980-12-15 00:00:00,0.094211,0.094211,0.093781,0.093781,175884800
1980-12-16 00:00:00,0.087328,0.087328,0.086898,0.086898,105728000
1980-12-17 00:00:00,0.089049,0.089479,0.089049,0.089049,86441600
1980-12-18 00:00:00,0.09163,0.092061,0.09163,0.09163,73449600
…,…,…,…,…,…
2024-10-15 00:00:00,233.610001,237.490005,232.369995,233.850006,64751400
2024-10-16 00:00:00,231.600006,232.119995,229.839996,231.779999,34082200
2024-10-17 00:00:00,233.429993,233.850006,230.520004,232.149994,32993800


In [ ]:
PRELUDE='''"""
Polars Expression Factory Methods
"""

# Do not edit! This file was generated.

import polars as pl

from typing import TypeAlias

from mintalib import core
from mintalib.model.expression import wrap_expression

IntoExpr: TypeAlias = pl.Expr | str | None

CLOSE = pl.col('close')
OHLC = pl.struct(['open', 'high', 'low', 'close'])



'''

exec(PRELUDE)

In [4]:


@wrap_expression(core.calc_sma)
def sma(src: IntoExpr, period: int) -> pl.Expr: ...


prices.select(
    sma(CLOSE, period=20).alias('sma20'),
)


sma20
f64
null
null
null
null
null
…
227.524
228.0785
228.2425


In [5]:
@wrap_expression(core.calc_macd)
def macd(src: IntoExpr = None): ...

prices.select(
    macd('close').struct.unnest()
)


macd,macdsignal,macdhist
f64,f64,f64
null,null,null
null,null,null
null,null,null
null,null,null
null,null,null
…,…,…
1.815958,1.313965,0.501993
1.941114,1.439395,0.501719
2.046565,1.560829,0.485736


In [6]:
@wrap_expression(core.calc_atr)
def atr(period: int, src: IntoExpr = None): ...

prices.select(atr(14))

atr
f64
null
null
null
null
null
…
4.516121
4.479971
4.39783


In [7]:
import inspect

from mintalib.builder import annotate_parameter

class Symbol(str):
    def __repr__(self):
        return self

s = Symbol("pl.Expr")
s

def make_signature(calc_func):
    """creates function signature from core function"""
    sig = inspect.signature(calc_func)
    
    new_params = []
    for param in sig.parameters.values():
        if param.name in ("prices", "series"):
            continue

        param = annotate_parameter(param)
        new_params.append(param)

    src = inspect.Parameter(
        name="src",
        default=None,
        kind=inspect.Parameter.KEYWORD_ONLY,
        annotation=Symbol("IntoExpr")
    )
    new_params.append(src)

    newsig = sig.replace(parameters=new_params, return_annotation=Symbol("pl.Expr"))

    return newsig

sig = make_signature(core.calc_sma)
print(sig)



(period: int, *, src: IntoExpr = None) -> pl.Expr


In [8]:

def make_expression(calc_func):
    cname = f"core.{calc_func.__name__}"
    fname = calc_func.__name__.removeprefix("calc_")
    signature = make_signature(calc_func)

    buffer = f"@wrap_expression({cname})\n"
    buffer += f"def {fname}{signature}: ...\n"
    return buffer


for cf in core.calc_sma, core.calc_macd, core.calc_atr:  
    code = make_expression(cf)
    print(code)



@wrap_expression(core.calc_sma)
def sma(period: int, *, src: IntoExpr = None) -> pl.Expr: ...

@wrap_expression(core.calc_macd)
def macd(n1: int = 12, n2: int = 26, n3: int = 9, *, src: IntoExpr = None) -> pl.Expr: ...

@wrap_expression(core.calc_atr)
def atr(period: int = 14, *, src: IntoExpr = None) -> pl.Expr: ...



In [9]:
def core_functions(exclude = ("calc_eval",)):
    names = sorted(k for k, v in vars(core).items() if k.startswith("calc_") and callable(v))
    if exclude:
        names = [n for n in names if n not in exclude]
    return names

core_functions()

['calc_abs',
 'calc_adx',
 'calc_alma',
 'calc_atr',
 'calc_avgprice',
 'calc_bbands',
 'calc_bbp',
 'calc_bbw',
 'calc_bop',
 'calc_cci',
 'calc_clag',
 'calc_cmf',
 'calc_crossover',
 'calc_crossunder',
 'calc_curve',
 'calc_dema',
 'calc_diff',
 'calc_dmi',
 'calc_ema',
 'calc_exp',
 'calc_flag',
 'calc_hma',
 'calc_kama',
 'calc_keltner',
 'calc_ker',
 'calc_lag',
 'calc_log',
 'calc_lroc',
 'calc_macd',
 'calc_macdv',
 'calc_mad',
 'calc_mav',
 'calc_max',
 'calc_mdi',
 'calc_mfi',
 'calc_midprice',
 'calc_min',
 'calc_natr',
 'calc_pdi',
 'calc_ppo',
 'calc_price',
 'calc_qsf',
 'calc_rma',
 'calc_roc',
 'calc_rsi',
 'calc_rvalue',
 'calc_sar',
 'calc_shift',
 'calc_sign',
 'calc_slope',
 'calc_sma',
 'calc_stdev',
 'calc_step',
 'calc_stoch',
 'calc_streak',
 'calc_sum',
 'calc_tema',
 'calc_trange',
 'calc_tsf',
 'calc_typprice',
 'calc_updown',
 'calc_wclprice',
 'calc_wma']

In [10]:
from pprint import pformat

import importlib.util

def make_expressions(cnames=None):
    if cnames is None:
        cnames = core_functions()

    output = PRELUDE

    fnames = []

    for cname in cnames:
        func = getattr(core, cname)
        name = cname.removeprefix("calc_")
        code = make_expression(func)
        fnames.append(name)
        output += code + "\n"

    return output

output = make_expressions()

def new_module(name: str, code: str = None):
    spec = importlib.util.spec_from_loader(name, None)
    module = importlib.util.module_from_spec(spec)
    if code:
        exec(code, module.__dict__)
    return module

expressions = new_module(f"{PACKAGE}.expressions", output)


In [11]:
prices.select(expressions.sma(period=5))


sma
f64
null
null
null
null
0.09206
…
230.256
230.704001
231.326001


In [12]:
module = new_module(f"{PACKAGE}.expressions", output)

outfile = PKGDIR / "expressions.py"

print(f"Updating {outfile.name} ...")

outfile.write_text(output)

Updating expressions.py ...


6980

In [13]:
args = "*", 10, 20
src, args = args[0], args[1:]
src, args

('*', (10, 20))

In [14]:
prices.select(pl.struct(pl.col(['high', 'low'])))

high
struct[2]
"{0.099373,0.098943}"
"{0.094211,0.093781}"
"{0.087328,0.086898}"
"{0.089479,0.089049}"
"{0.092061,0.09163}"
…
"{237.490005,232.369995}"
"{232.119995,229.839996}"
"{233.850006,230.520004}"


In [15]:
sma.__doc__

'\n    Simple Moving Average\n    \n    Args:\n        period (int) : time period, required\n    '